In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss, confusion_matrix
from sklearn.impute import KNNImputer

In [3]:
import lightgbm as lgb
from tqdm import tqdm

In [4]:
import optuna

In [5]:
from imblearn.over_sampling import SMOTE

In [6]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')

In [7]:
train.rename(columns = {"Class" : "target"}, inplace = True)
train.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,target
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [8]:
train['target'].value_counts()

0    509
1    108
Name: target, dtype: int64

In [9]:
one_hot_encoded = pd.get_dummies(train[['EJ']])
print(one_hot_encoded.head())

   EJ_A  EJ_B
0     0     1
1     1     0
2     0     1
3     0     1
4     0     1


In [10]:
for column in one_hot_encoded.columns:
   train.insert(len(train.columns), column , one_hot_encoded.loc[:, column])

In [11]:
train.drop(['EJ', 'Id'], axis=1, inplace=True)
target_column = train["target"]
train.drop(["target"], inplace=True, axis=1) # Remove target column for training the data


In [12]:
imputer = KNNImputer(n_neighbors=2)
imputed_to_fit_on_test = imputer.fit(train)
imputed_data = imputer.fit_transform(train)

In [13]:
train = pd.DataFrame(imputed_data, columns = train.columns)

In [14]:
scaled_to_fit_on_test =  MinMaxScaler().fit(train)

def standardizationMinMax(dataset) :
    scaler = MinMaxScaler()
    return scaler.fit_transform(dataset)

In [15]:
train = pd.DataFrame(standardizationMinMax(train) , columns = train.columns)

In [16]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(train, target_column)

In [17]:
train = X_resampled
target_column = y_resampled

In [18]:
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
one_hot_encoded = pd.get_dummies(test[['EJ']])
print(one_hot_encoded.head())
test.drop(['EJ', 'Id'], axis=1, inplace=True)
for column in one_hot_encoded.columns:
   test.insert(len(test.columns), column , one_hot_encoded.loc[:, column])
if "EJ_B" not in test.columns:
    test["EJ_B"] = 0
if "EJ_A" not in test.columns:
    test["EJ_A"] = 1
    
test = pd.DataFrame(imputed_to_fit_on_test.transform(test), columns=test.columns) 


   EJ_A
0     1
1     1
2     1
3     1
4     1


In [19]:

def objective() :
    params = {'learning_rate': 0.00010528656953854453,
 'n_estimators': 465,
 'max_depth': 4,
 'num_leaves': 742,
 'subsample': 0.5801452942400647,
 'colsample_bytree': 0.6255791094574136,
 'min_child_samples': 73,
 'min_split_gain': 0.5803609086856948,
 'reg_alpha': 0.38743183795216396,
 'reg_lambda': 0.030194584054500706}
    
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    global model
    model = lgb.LGBMClassifier(**params, random_state=42, n_jobs=-1)
    
    scores = []
    for train_idx, test_idx in tqdm(skf.split(train,target_column)):
        x_train, x_test = train.loc[train_idx], train.loc[test_idx] 
        y_train, y_test = target_column.loc[train_idx] , target_column.loc[test_idx]

        model.fit(x_train, y_train)
        
        predictions = model.predict(x_test)
        score = accuracy_score(y_test, predictions)
        
        scores.append(score)
        
    
    return np.mean(scores)
    


In [20]:
objective()

10it [00:10,  1.05s/it]


0.8899437002523782

In [21]:
# Make predictions on the test dataset
prediction = model.predict(test)

prediction

array([0, 0, 0, 0, 0])

In [22]:
y_proba = model.predict_proba(test)

In [23]:
# Calculate the probability for the positive class
positive_class_probability = y_proba[:, 1]

# Calculate the probability for the negative class
negative_class_probability = y_proba[:, 0]

# Print the probabilities
print("Positive class probability:", positive_class_probability)
print("Negative class probability:", negative_class_probability)

Positive class probability: [0.48538102 0.48538102 0.48538102 0.48538102 0.48538102]
Negative class probability: [0.51461898 0.51461898 0.51461898 0.51461898 0.51461898]


In [24]:
test_2 = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')

In [25]:

submissions = pd.DataFrame(
    {"Id": test_2['Id'], "class_0": negative_class_probability, "class_1": 1 - negative_class_probability} 
)

In [26]:
submissions.to_csv("submission.csv", index=False)

In [27]:
pd.read_csv('/kaggle/working/submission.csv').head()

,Id,class_0,class_1
0,00eed32682bb,0.514619,0.485381
1,010ebe33f668,0.514619,0.485381
2,02fa521e1838,0.514619,0.485381
3,040e15f562a2,0.514619,0.485381
4,046e85c7cc7f,0.514619,0.485381
